In [1]:
using Revise
using VariPEPS
using OptimKit
using TensorKit
using JLD2

In [2]:
#= specify the Bond-dimension d, the environment-bond-dimension χ. Further one may specify the model that one wants to test, the 
projectors that should be used in the CTMRG-routine, as well as Hamiltonian parameters. =#
χ = 9
d = 2


#=Here specify the unit cell. For this we define an array "Pattern_arr" that represents the unit cell structure.
The structure is [x-coodinate, y-coordinate]. If two positions within the Pattern_arr are filled with the same number, we assume the tensors at these positions to be identical.
The choice below corresponds to a unit cell like:   A   B 
                                                    B   A=#
Pattern_arr = Array{Any}(undef,3,3)

Pattern_arr[1,1] = 1
Pattern_arr[2,1] = 2
Pattern_arr[3,1] = 3


Pattern_arr[1,2] = 2
Pattern_arr[2,2] = 3
Pattern_arr[3,2] = 1

Pattern_arr[1,3] = 3
Pattern_arr[2,3] = 1
Pattern_arr[3,3] = 2


#=For our first example we are going to look at the Heisenberg-antiferromagnet on the square lattice.
Here we specify the keywords corresponding to this choice of model and lattice on in a tuple.

Say more about the possible keywords
=# 
keywords = (lattice = :triangular, model = :Heisenberg_triangular, Ham_parameters = (Jxy = 1, Jdiag = 1, h = 0, dir = [0,0,0]), Projector_type = :half, Space_type = ℂ, svd_type = :envbond)

(lattice = :triangular, model = :Heisenberg_triangular, Ham_parameters = (Jxy = 1, Jdiag = 1, h = 0, dir = [0, 0, 0]), Projector_type = :half, Space_type = ℂ, svd_type = :envbond)

In [3]:
#= 
this functiion initializes the PEPS-tensors. 
The first input is the Bond dimension, 
the second input is the dimension of the local Hilbert space, --> This is 2 in the current example, as we are dealing with a spin 1/2 system.
the third one specified how many tensors need to be generated. --> In this example 2 since we have two inequivalent tensors in the unit cell.

What is created here is just an array of TensorMaps with the right number of legs with the desired dimensionality.

Note that we can also enforce local symmetries and only optimize on the parametes for the local tensors that have this symmetry. --> Add mini example later.

=#
loc_in = initialize_PEPS(d, 2, 3; seed = 1236, lattice = :square, Number_type = ComplexF64, Space_type = ℂ); #change the code such that if we use real numbers we use "normalization_convention_without_phase"
  

3-element Vector{Any}:
 TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^2)')):
[:, :, 1, 1, 1] =
 0.24197647958244134 + 0.22935586247095718im  …   0.2824527775439144 + 0.08377178393398838im
 0.15384512414632187 - 0.06724498698693439im     0.11222501246349147 + 0.09544425402670877im

[:, :, 2, 1, 1] =
  0.3149769054298578 + 0.2882560574048658im  …  -0.18425696192857632 + 0.231520153406199im
 0.18415019217159873 - 0.2775547323846692im        0.523192047602709 + 0.16614131432658397im

[:, :, 1, 2, 1] =
 0.16368141811228704 + 0.053712454233148285im  …    -0.1093908032856745 - 0.3251634270977488im
 0.12920310906336896 + 0.12403026075019324im      -0.041642605325531115 + 0.14564262701389508im

[:, :, 2, 2, 1] =
   0.0901138229997678 + 0.14410342435139045im  …  -0.10312525756663206 - 0.13563529974708458im
 0.013203413196373581 + 0.2751706273878261im      -0.18556854570149534 - 0.10061367942522378im

[:, :, 1, 1, 2] =
 0.28939818946506557 + 0.19256428917136534im  …  0.14528156958510718 + 0.27158583530

In [5]:
#the ctmrg function performs a CTM-RG procedure. -- Put more details on elementwise convergence and so on.
#= note that in some cases, the GKL algorithm, that is used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms. If one does not want to use the GKL algorithm at all, we can use the keyword svd_type = :envbond
to just use conventional SVD algorithms.=#

#=In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.=#

ctmrg_res = ctmrg(loc_in, χ, Pattern_arr; keywords..., observ = true, conv_info = false, maxiter = 200, adjust_χ = false)

(0.013942785346161746, Any[-0.0321760841590479 + 2.0931909682940587e-17im, -0.05843358077593848 - 1.4431444365651573e-17im, -0.1120000543849041 + 4.006989367583578e-17im, 0.023761155416579163 + 3.603629367706816e-17im, -0.048509609954064295 + 4.122267853383184e-17im, 0.07394068462586516 - 1.6378893016944674e-17im, -0.02759039597779776 + 1.6986514863897894e-17im, -0.17633092806365333 + 6.792738426979449e-17im, -0.07733651718929452 + 8.489098369684297e-18im])

In [14]:
stagmag = 0
magn_vec = real.(ctmrg_res[2])
stag_mag_vec = (magn_vec[1:3] - magn_vec[4:6]) ./ 2
stagmag = norm(stag_mag_vec)

0.3811010897844127

For a state, obtained after a few optimization steps of the Heisenberg model we look at the truncation errors as 
a function of the bond dimension - with the corresponding energy. The upper block is for the "fullfishman"-projectors
while the lower block is for the "half"-projectors. We conclude from the convergence of the energy-expectation values
that a maximal trunctation error of ~10^-3 is a reasonable starting point. It can always be chosen more stricly.

E = -0.6603737287187401 @ Bond_env = 80, maximum(trunc_sv_arr) = 5.975686042268443e-5 
E = -0.6603737358938596 @ Bond_env = 60, maximum(trunc_sv_arr) = 0.00012481759922763688
E = -0.6603737799922393 @ Bond_env = 40, maximum(trunc_sv_arr) = 0.00031194042937930866
E = -0.6603736187795066 @ Bond_env = 30, maximum(trunc_sv_arr) = 0.0006405720791402293
E = -0.6603724597506875 @ Bond_env = 26, maximum(trunc_sv_arr) = 0.0009362616891090525
E = -0.6603639207050201 @ Bond_env = 20, maximum(trunc_sv_arr) = 0.0020002907780533145
E = -0.6603793732337668 @ Bond_env = 16, maximum(trunc_sv_arr) = 0.003927764975811235


E = -0.6603737141782002 @ Bond_env = 80, maximum(trunc_sv_arr) = 8.859010589321488e-5
E = -0.6603736039935315 @ Bond_env = 60, maximum(trunc_sv_arr) = 0.00018689030517925998
E = -0.6603739415374238 @ Bond_env = 40, maximum(trunc_sv_arr) = 0.0004921465970557896
E = -0.6603749442733124 @ Bond_env = 30, maximum(trunc_sv_arr) = 0.0011953646129798726
E = -0.6603675868543281 @ Bond_env = 26, maximum(trunc_sv_arr) = 0.00172234383475717
E = -0.6603589442727148 @ Bond_env = 20, maximum(trunc_sv_arr) = 0.003446092986088426
E = -0.6603566068572576 @ Bond_env = 16, maximum(trunc_sv_arr) = 0.005807880961672133

In [6]:
#the function energy_and_gradient is needed for the variational optimization. It calculates the gradient at the fixed point of the CTM-RG iteration.
#= note that in some cases, the GKL algorithm, that is used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms. If one does not want to use the GKL algorithm at all, we can use the keyword svd_type = :envbond
to just use conventional SVD algorithms.=#

#=In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.=#

#=Now there are two ways to call the "energy_and_gradient" function.
you can use:
THIS IS THE STANDART CASE THAT SHOULD BE USED IF WE DONT WANT TO INCREASE χ DYNAMICALLY IN THE OPTIMIZATION
1. e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)
This version is the standart version. It takes a (local PEPS, environment bond dimension, a pattern array for the unit cell; a bunch of keywords)


THIS IS JUST A WRAPPER AROUND THE FIRST VERSION WHICH RETURNS THE ADJUSTED χ VALUE IF IT IS INCREASED. THIS IS NEEDED FOR DYNAMICAL χ INCREASE.
2. e, gr = energy_and_gradient(loc_in, χ, Pattern_arr, keywords; adjust_χ = false)
In this version the only thing to be specified by keyword is "adjust_χ" which is either false of (max_χ, tol)
TWO IMPORTANT CAVEATS:
a) in this case χ needs to be an array as [χ]. This is needed for dynamical increase of χ (it needs to be a mutable type e.g. array)
b) We can still specify the options that were previously specified by keywords. We just pass a tuple (named tuple actually) that contains the keywords we want
to specify.
=#
e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)

Info: it took 16 CTMRG steps to converge the SV to 1.0e-6
Info: it took 20 CTMRG steps to converge the environments element wise to 1e-6


(0.01394278685298784, TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^2)')):
[:, :, 1, 1, 1] =
 -0.0069089096230851976 + 0.0004711254902462122im  …  -0.014336783596520726 + 0.00152944332250085im
   0.006241430751029748 - 0.011435331259022036im      -0.004073898741435403 + 0.004767314409262334im

[:, :, 2, 1, 1] =
 0.0042267610633688665 - 0.0027173075281253243im  …  -0.004234433783717921 - 0.003988794848123747im
 -0.000338303245678785 + 0.015899372614988486im      -0.002999013200557303 - 0.004066109577430955im

[:, :, 1, 2, 1] =
  0.006877275552056742 - 0.013300431671654681im  …   0.012873620147575733 - 0.002451935781730752im
 -0.013775202230598357 + 0.007494458435843226im     0.0013864544295594825 + 0.012012712692391804im

[:, :, 2, 2, 1] =
 0.0026142365086199762 + 0.017582260209268646im  …  -0.043440081722488984 + 0.029776148841592895im
   0.00787097365331087 - 0.005734721545596548im      0.005329271491093261 + 0.008301998520257616im

[

In [7]:
#for the optimization we need the following two functions:

function _finalize!(x,f,g, numiter)
    flush(stdout)
    flush(stderr)

    
    #=if numiter%2 == 1
        file = jldopen("PEPS_during_optimization.jld2", "a+")
        write(file, "PEPS_at_opt_step_$(numiter)", x)
        close(file)
    end=#
     
    
    return x, f, g
end

function _inner(x, ξ_1, ξ_2)
    return dot(real(ξ_1), real(ξ_2)) + dot(imag(ξ_1), imag(ξ_2))
end

_inner (generic function with 1 method)

In [12]:
#we specify the parameters that are used in the optimization.

optimparas1 = (gradtol = 1e-5, verbosity = 2, maxiter = 100, linesearch = HagerZhangLineSearch(; c₁ = 1e-4, c₂ = 0.9, maxiter = 40))
kwargs = (finalize! = _finalize!, inner = _inner)


(finalize! = _finalize!, inner = _inner)

In [13]:
x, e, gr, numfg, normgradhistory = optimize(x -> energy_and_gradient(x, χ, Pattern_arr; keywords...), loc_in, LBFGS(15; optimparas1... ); kwargs... )


Info: it took 16 CTMRG steps to converge the SV to 1.0e-6
Info: it took 20 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: initializing with f = 0.013942786853, ‖∇f‖ = 1.6761e-01
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:35


Info: it took 18 CTMRG steps to converge the SV to 1.0e-6
Info: it took 23 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter    1: f = -0.243992337291, ‖∇f‖ = 3.8281e-01, α = 1.00e+00, m = 0, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 13 CTMRG steps to converge the SV to 1.0e-6
Info: it took 16 CTMRG steps to converge the environments element wise to 1e-6


...

Info: it took 9 CTMRG steps to converge the SV to 1.0e-6
Info: it took 12 CTMRG steps to converge the environments element wise to 1e-6
Info: it took 9 CTMRG steps to converge the SV to 1.0e-6
Info: it took 12 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter   90: f = -0.526760351550, ‖∇f‖ = 1.9388e-05, α = 4.54e-01, m = 15, nfg = 2
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 9 CTMRG steps to converge the SV to 1.0e-6
Info: it took 12 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: converged after 91 iterations: f = -0.526760365575, ‖∇f‖ = 8.4629e-06
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


(TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^2)')):
[:, :, 1, 1, 1] =
  0.05798040858226042 + 0.527380164627356im    …  0.43985153016098594 + 0.5600027877072286im
 0.017310601464977718 - 0.17937710535412577im      0.5503989117451495 + 0.48168237752600135im

[:, :, 2, 1, 1] =
  -0.6554050244052784 + 0.321030832211293im    …  0.3451296221429914 + 0.3432431747362308im
 -0.34097250295247594 - 0.09432284696512705im     0.3026377679554408 + 0.3336075799186322im

[:, :, 1, 2, 1] =
  0.5878102310327132 - 0.20522364646679006im  …  -0.014818526067571685 - 0.6134682124600108im
 0.48343151589456684 + 0.5420994811845905im        0.06719563571507782 - 0.5088600277215156im

[:, :, 2, 2, 1] =
 0.8482190904586374 + 0.5157288178752387im  …  0.8451519116405263 - 1.5376876054857613im
 -0.669057422948472 + 1.4274589299388056im      0.582595500680413 - 1.289598572979056im

[:, :, 1, 1, 2] =
 -0.04113058964980991 + 0.5021021253888088im   …  0.9227713380569